In [1]:
import pandas as pd
import os

# Charger le DataFrame INVENTAIRE2
#inventaire2_df = pd.read_excel("consolidated_data.xlsx")


# Chemin du répertoire contenant les fichiers Excel
input_directory = 'C:\\Users\\Administrateur\\Desktop\\code\\input'

# Liste pour stocker les DataFrames de chaque fichier
data_frames = []

# Parcourir tous les fichiers Excel dans le répertoire
for filename in os.listdir(input_directory):
    if filename.endswith('.xlsx') and not filename.startswith('~$'):
        file_path = os.path.join(input_directory, filename)
        try:
            # Lire le fichier Excel dans un DataFrame
            df = pd.read_excel(file_path)
            # Ajouter le DataFrame à la liste
            data_frames.append(df)
        except PermissionError:
            print(f"Ignoré : {filename} (Fichier verrouillé)")

# Concaténer les DataFrames en un seul DataFrame
consolidated_df = pd.concat(data_frames, ignore_index=True)


inventaire2_df = consolidated_df 

# Renommer la colonne "Famille/Produit" en "Item"
inventaire2_df.rename(columns={"Famille/Produit": "Item"}, inplace=True)

# Charger le DataFrame de correspondance
correspondances_df = pd.read_excel("correspondances.xlsx")

# Créer les nouvelles colonnes "Catégorie" et "Sous-catégorie"
inventaire2_df["Catégorie"] = ""
inventaire2_df["Sous-catégorie"] = ""

# Remplir les colonnes "Catégorie" et "Sous-catégorie" en utilisant la correspondance
for index, row in inventaire2_df.iterrows():
    item = row["Item"]
    matching_row = correspondances_df[correspondances_df["Item"] == item]
    if not matching_row.empty:
        inventaire2_df.at[index, "Catégorie"] = matching_row["Catégorie"].values[0]
        inventaire2_df.at[index, "Sous-catégorie"] = matching_row["Sous-catégorie"].values[0]

# Remplacer les "NaN" dans la colonne "Total HT" par 0
inventaire2_df["Total HT"].fillna(0, inplace=True)

# Réorganiser les colonnes
column_order = ["Catégorie", "Sous-catégorie", "Item","Qté", "Offert","Offert formule","Total Qté","Total TTC","Coût",
                "Total remise","TTC remisé","Total HT","Mois","Années"]
inventaire2_df = inventaire2_df[column_order]

# Supprimer les lignes commençant par "Total..." ou "TOTAL..."
inventaire2_df = inventaire2_df[~inventaire2_df["Item"].str.startswith("Total", na=False)]
inventaire2_df = inventaire2_df[~inventaire2_df["Item"].str.startswith("TOTAL", na=False)]

# Supprimer les lignes ayant au moins trois NaN
inventaire2_df = inventaire2_df.dropna(thresh=inventaire2_df.shape[1] - 3)

# Supprimer les lignes vides
inventaire2_df = inventaire2_df.dropna(how="all")

# Réinitialiser les index
inventaire2_df.reset_index(drop=True, inplace=True)

# Calculer la somme de Total Qté par Sous-Catégorie
sous_cat_sum = inventaire2_df.groupby('Sous-catégorie')['Total Qté'].transform('sum')

# Calculer la colonne "Quantité Absolue"
inventaire2_df['Quantité Absolue'] = inventaire2_df['Total Qté'] / sous_cat_sum*100

# Calculer la colonne "Quantité Relative"
total_sum = inventaire2_df['Total Qté'].sum()

inventaire2_df['Quantité Relative'] = inventaire2_df['Total Qté'] / sous_cat_sum

# Afficher le DataFrame résultant
#inventaire2_df



# Chemin du répertoire où vous voulez enregistrer le fichier Excel
output_directory = 'C:\\Users\\Administrateur\\Desktop\\code\\inputcons'

# Vérifier si le répertoire existe, sinon le créer
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Chemin complet du fichier de sortie
output_path = os.path.join(output_directory, 'BD.xlsx')

# Enregistrer le DataFrame en fichier Excel
inventaire2_df.to_excel(output_path, index=False)

print("Fichier Excel enregistré avec succès sous le nom 'BD.xlsx'.")

Fichier Excel enregistré avec succès sous le nom 'BD.xlsx'.


In [2]:
inventaire2_df

,Catégorie,Sous-catégorie,Item,Qté,Offert,Offert formule,Total Qté,Total TTC,Coût,Total remise,TTC remisé,Total HT,Mois,Années,Quantité Absolue,Quantité Relative
0,Drink,SOFTS,REDBULL,0,14.0,0.0,14,0 CFA,0 CFA,0 CFA,0 CFA,0.0,Avril,2023.0,0.267686,0.002677
1,Drink,BIERES,BEAUFORT,32,1.0,0.0,33,48 000 CFA,0 CFA,0 CFA,48 000 CFA,48000.0,Avril,2023.0,1.535598,0.015356
2,Drink,BIERES,CASTEL,1,0.0,0.0,1,1 500 CFA,0 CFA,0 CFA,1 500 CFA,1500.0,Avril,2023.0,0.046533,0.000465
3,Drink,BIERES,DESPERADOS,13,0.0,0.0,13,32 500 CFA,0 CFA,0 CFA,32 500 CFA,32500.0,Avril,2023.0,0.604933,0.006049
4,Drink,BIERES,DJAMA,21,0.0,0.0,21,31 500 CFA,0 CFA,0 CFA,31 500 CFA,31500.0,Avril,2023.0,0.977199,0.009772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,Drink,WHISKY,JACK DANIELS BTL,7,4.0,0.0,11,315 000 CFA,0 CFA,45 000 CFA,270 000 CFA,270000.0,Juillet,2023.0,2.528736,0.025287
584,Drink,WHISKY,JACK DANIELS GENTLEMEN BTL,1,0.0,0.0,1,60 000 CFA,0 CFA,10 000 CFA,50 000 CFA,50000.0,Juillet,2023.0,0.229885,0.002299
585,Drink,WHISKY,JACK DANIELS HONEY BTL,6,2.0,0.0,8,270 000 CFA,0 CFA,40 000 CFA,230 000 CFA,230000.0,Juillet,2023.0,1.83908,0.018391
586,Drink,WHISKY,RED LABEL,27,0.0,0.0,27,81 000 CFA,0 CFA,0 CFA,81 000 CFA,81000.0,Juillet,2023.0,6.206897,0.062069
